# Trend Indicators - Apple Stock 2023-2025

Testing trend indicators: SMA, EMA, VWAP.

In [2]:
import sys
from pathlib import Path

import plotly.graph_objects as go
import yfinance as yf
from plotly.subplots import make_subplots

sys.path.insert(0, str(Path("../..").resolve()))

from indicators.trend import calculate_ema, calculate_sma, calculate_vwap

In [3]:
# Fetch Apple data
df = yf.download("AAPL", start="2023-01-01", end="2025-10-01", auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330643,129.079559,122.443158,128.468186,112117500
1,2023-01-04,124.602707,126.870724,123.340509,125.125335,89113600
2,2023-01-05,123.281334,125.993089,123.024955,125.361991,80962700
3,2023-01-06,127.817352,128.478033,123.153137,124.257564,87754700
4,2023-01-09,128.339996,131.554669,128.083618,128.655553,70790800


## 1. SMA - Simple Moving Average

Simple average of past N periods. Smooths noise, identifies trend direction. Price above SMA = uptrend, below = downtrend.

In [4]:
df["sma_20"] = calculate_sma(df, period=20)
df["sma_50"] = calculate_sma(df, period=50)
df["sma_200"] = calculate_sma(df, period=200)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}))
fig.add_trace(go.Scatter(x=df["date"], y=df["sma_20"], name="SMA 20", line={"color": "blue", "dash": "dash"}))
fig.add_trace(go.Scatter(x=df["date"], y=df["sma_50"], name="SMA 50", line={"color": "orange", "dash": "dash"}))
fig.add_trace(go.Scatter(x=df["date"], y=df["sma_200"], name="SMA 200", line={"color": "red", "dash": "dash"}))

fig.update_layout(height=600, title_text="Simple Moving Averages", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("SMA = Simple average of past N periods")
print("20-day: short-term trend")
print("50-day: intermediate trend")
print("200-day: long-term trend")

SMA = Simple average of past N periods
20-day: short-term trend
50-day: intermediate trend
200-day: long-term trend


## 2. EMA - Exponential Moving Average

Exponentially weighted average giving more weight to recent prices. Responds faster to price changes than SMA.

In [5]:
df["ema_12"] = calculate_ema(df, period=12)
df["ema_26"] = calculate_ema(df, period=26)
df["ema_50"] = calculate_ema(df, period=50)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_12"], name="EMA 12", line={"color": "green"}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_26"], name="EMA 26", line={"color": "blue"}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_50"], name="EMA 50", line={"color": "red"}))

fig.update_layout(height=600, title_text="Exponential Moving Averages", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("EMA = Exponentially weighted average (more weight on recent data)")
print("EMA responds faster to price changes than SMA")

EMA = Exponentially weighted average (more weight on recent data)
EMA responds faster to price changes than SMA


## 3. VWAP - Volume Weighted Average Price

Average price weighted by volume. Used by institutions to assess execution quality. Price above VWAP = bullish, below = bearish.

In [6]:
df["vwap"] = calculate_vwap(df)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price vs VWAP", "Volume")
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}), row=1, col=1)
fig.add_trace(go.Scatter(x=df["date"], y=df["vwap"], name="VWAP", line={"color": "purple", "width": 2}), row=1, col=1)

fig.add_trace(go.Bar(x=df["date"], y=df["volume"], name="Volume", marker_color="lightblue"), row=2, col=1)

fig.update_layout(height=700, title_text="Volume Weighted Average Price")
fig.show()

print("VWAP = Average price weighted by volume")
print("Used by institutions to assess execution quality")
print("Price above VWAP = bullish, below = bearish")

VWAP = Average price weighted by volume
Used by institutions to assess execution quality
Price above VWAP = bullish, below = bearish


## SMA vs EMA Comparison

In [7]:
# Compare SMA 50 vs EMA 50
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1}))
fig.add_trace(go.Scatter(x=df["date"], y=df["sma_50"], name="SMA 50", line={"color": "blue", "dash": "dash"}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_50"], name="EMA 50", line={"color": "red"}))

fig.update_layout(height=600, title_text="SMA vs EMA Comparison (50-period)", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("EMA (red) reacts faster to price changes than SMA (blue)")

EMA (red) reacts faster to price changes than SMA (blue)


## Combined View - All Trend Indicators

In [ ]:
fig = go.Figure()

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}))

# SMAs
fig.add_trace(
    go.Scatter(x=df["date"], y=df["sma_20"], name="SMA 20", line={"color": "lightblue", "dash": "dash", "width": 1})
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["sma_50"], name="SMA 50", line={"color": "blue", "dash": "dash", "width": 1.5})
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["sma_200"], name="SMA 200", line={"color": "darkblue", "dash": "dash", "width": 2})
)

# EMAs
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_12"], name="EMA 12", line={"color": "lightgreen", "width": 1}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_26"], name="EMA 26", line={"color": "green", "width": 1.5}))
fig.add_trace(go.Scatter(x=df["date"], y=df["ema_50"], name="EMA 50", line={"color": "darkgreen", "width": 2}))

# VWAP
fig.add_trace(go.Scatter(x=df["date"], y=df["vwap"], name="VWAP", line={"color": "purple", "width": 2, "dash": "dot"}))

fig.update_layout(
    height=700, title_text="Trend Indicators - Complete Overview", xaxis_title="Date", yaxis_title="Price"
)
fig.show()

print("\n=== Trend Summary ===")
print(f"Price vs SMA 200: {'Above' if df['close'].iloc[-1] > df['sma_200'].iloc[-1] else 'Below'}")
print(f"Price vs EMA 50: {'Above' if df['close'].iloc[-1] > df['ema_50'].iloc[-1] else 'Below'}")
print(f"Price vs VWAP: {'Above' if df['close'].iloc[-1] > df['vwap'].iloc[-1] else 'Below'}")
print("\nMoving averages smooth noise and identify trend direction.")
print("Crossovers can signal trend changes (e.g., Golden Cross, Death Cross).")